In [3]:
import os
import pandas as pd
import seaborn as sns

Считывание данных

In [4]:
#А теперь считаем и объединим информацию по пользователям и feed_data с условием
#чтобы для каждого пользователя количество действий было равно определенному числу

In [5]:
user_behavior = pd.read_sql(
    """SELECT u.*, x.*
    FROM user_data u
    LEFT JOIN LATERAL (SELECT *,
    ROW_NUMBER() OVER(PARTITION BY user_id) AS rownum
    FROM feed_data) x on u.user_id = x.user_id
    WHERE x.rownum <= 5""",
       con="postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
         "postgres.lab.karpov.courses:6432/startml")

In [6]:
post_text_df = pd.read_sql(
    """SELECT *
    FROM post_text_df
    """,
       con="postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
         "postgres.lab.karpov.courses:6432/startml")

In [7]:
#Создадим на всякий случай копию датафрейма, чтобы какждый раз не грузить по новой данные
user_behavior_copy = user_behavior.copy()

In [8]:
#удалим технический столбец
user_behavior_copy = user_behavior_copy.drop("rownum", axis=1)

In [9]:
#Удалим повторяющиеся колонки post_id
user_behavior_copy = user_behavior_copy.loc[:,~user_behavior_copy.columns.duplicated()].copy()

In [10]:
user_behavior_copy.head(10)

,user_id,gender,age,country,city,exp_group,os,source,timestamp,post_id,action,target
0,200,1,34,Russia,Degtyarsk,3,Android,ads,2021-11-13 19:30:21,2067,view,0
1,200,1,34,Russia,Degtyarsk,3,Android,ads,2021-11-13 19:30:50,5206,view,0
2,200,1,34,Russia,Degtyarsk,3,Android,ads,2021-11-13 19:33:21,5807,view,0
3,200,1,34,Russia,Degtyarsk,3,Android,ads,2021-11-13 19:36:15,6661,view,1
4,200,1,34,Russia,Degtyarsk,3,Android,ads,2021-11-13 19:38:59,6661,like,0
5,201,0,37,Russia,Abakan,0,Android,ads,2021-10-04 09:44:05,1033,view,0
6,201,0,37,Russia,Abakan,0,Android,ads,2021-10-04 09:45:34,6917,view,0
7,201,0,37,Russia,Abakan,0,Android,ads,2021-10-04 09:45:50,3991,view,0
8,201,0,37,Russia,Abakan,0,Android,ads,2021-10-04 09:46:35,4240,view,0
9,201,0,37,Russia,Abakan,0,Android,ads,2021-10-04 09:48:51,3270,view,0


In [10]:
user_behavior_copy.isna().sum()

user_id      0
gender       0
age          0
country      0
city         0
exp_group    0
os           0
source       0
timestamp    0
post_id      0
action       0
target       0
dtype: int64

In [ ]:
sns.catplot(x="gender", kind="count", col="target", data=user_behavior_copy, sharey=False)

In [ ]:
sns.catplot(x="age", kind="count", col="target", data=user_behavior_copy, sharey=False)

In [ ]:
sns.catplot(x="country", kind="count", col="target", data=user_behavior_copy, sharey=False)

In [ ]:
sns.catplot(x="os", kind="count", col="target", data=user_behavior_copy, sharey=False)

In [ ]:
sns.catplot(x="exp_group", kind="count", col="target", data=user_behavior_copy, sharey=False)

In [11]:
#для просмотров, где действие являлось лайком, проставим тоже 1 в таргет
user_behavior_copy.loc[(user_behavior_copy["action"] == "like"), "target"] = 1

In [26]:
#удалим пока что акшионы
user_behavior_copy = user_behavior_copy.drop("action", axis=1)

In [12]:
#удалим город, их слишком много, потом подумаем что делать с городом
user_behavior_copy = user_behavior_copy.drop("city", axis=1)

In [13]:
user_behavior_copy['timestamp'] = pd.to_datetime(user_behavior_copy['timestamp'])
user_behavior_copy['hour'] = user_behavior_copy.timestamp.dt.hour

In [ ]:
user_behavior_copy.head()

In [16]:
user_behavior_copy.nunique()

user_id      163205
gender            2
age              76
country          11
exp_group         5
os                2
source            2
timestamp    859644
post_id        6831
target            2
hour             19
dtype: int64

In [17]:
post_text_df.nunique()

post_id    7023
text       6924
topic         7
dtype: int64

In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
tf_idf = vectorizer.fit_transform(post_text_df["text"])

In [28]:
from sklearn.decomposition import TruncatedSVD

In [ ]:
svd = TruncatedSVD(n_components=200)
tf_idf_svd = svd.fit_transform(tf_idf)

In [ ]:
df_tf_idf_svd = pd.DataFrame(tf_idf_svd)

In [ ]:
type(df_tf_idf_svd)

In [ ]:
post_text_df = pd.concat([post_text_df, df_tf_idf_svd], axis=1)

In [ ]:
user_behavior_copy = user_behavior_copy.merge(post_text_df, on=["post_id"])


In [ ]:
user_behavior_copy.head()

In [ ]:
#пока что удалим текст, потом попробуем TF-IDF
user_behavior_copy = user_behavior_copy.drop("text", axis=1)

In [18]:
user_behavior_copy.set_index(["post_id", "user_id"], inplace=True)

In [44]:
user_behavior_copy.head(100)

,user_id,gender,age,country,exp_group,os,source,timestamp,post_id,action,target,hour
0,200,1,34,Russia,3,Android,ads,2021-11-13 19:43:33,1830,view,0,19
1,200,1,34,Russia,3,Android,ads,2021-11-13 19:44:06,1496,view,0,19
2,200,1,34,Russia,3,Android,ads,2021-11-13 19:44:48,1765,view,0,19
3,200,1,34,Russia,3,Android,ads,2021-11-13 19:46:12,992,view,1,19
4,200,1,34,Russia,3,Android,ads,2021-11-13 19:48:00,992,like,1,19
...,...,...,...,...,...,...,...,...,...,...,...,...
95,211,1,22,Belarus,0,iOS,ads,2021-10-16 08:20:46,6651,view,0,8
96,212,1,47,Russia,0,iOS,ads,2021-10-01 18:20:43,2877,view,0,18
97,212,1,47,Russia,0,iOS,ads,2021-10-01 18:23:01,4879,view,0,18
98,212,1,47,Russia,0,iOS,ads,2021-10-01 18:23:53,3434,view,0,18


In [ ]:
from sklearn.model_selection import TimeSeriesSplit
splitter = TimeSeriesSplit(n_splits=4)

In [ ]:
from catboost import CatBoostClassifier

In [ ]:
from sklearn.model_selection import train_test_split

from sklearn.metrics import classification_report

X = user_behavior_copy.drop('target', axis=1)
y = user_behavior_copy['target']

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    random_state=0,
                                                    shuffle=True,
                                                    test_size=0.2)

cboost = CatBoostClassifier(auto_class_weights="SqrtBalanced")

cboost.fit(X_train,
           y_train,
           cat_features=['country', "os", "source", "topic"] )

y_pred = cboost.predict(X_test)

print(classification_report(y_test, y_pred, digits=4))

In [25]:
cboost.feature_importances_

array([8.21985452e-01, 1.47580930e-01, 2.66468053e+00, 5.85517541e-01,
       9.50635676e-01, 5.87804324e-02, 0.00000000e+00, 1.18695391e+00,
       4.42887875e-01, 8.02452520e+01, 7.67521221e-01, 2.67448844e-01,
       1.45682647e-01, 2.54351748e-01, 1.17641187e-01, 3.13784617e-01,
       1.20137857e-01, 5.05505201e-02, 3.76261864e-02, 6.87166578e-02,
       1.03287751e-01, 1.48305821e-01, 1.24168147e-01, 7.37282585e-02,
       7.46292036e-02, 5.91468798e-02, 8.81132027e-02, 5.52782328e-02,
       5.60825852e-02, 4.84050973e-02, 5.82395527e-02, 9.40756191e-02,
       5.31732533e-02, 4.93619249e-02, 5.03985647e-02, 7.08216256e-02,
       4.76675904e-02, 7.90641869e-02, 3.89093966e-02, 6.13619426e-02,
       7.13503896e-02, 4.43702746e-02, 6.03589044e-02, 6.77997917e-02,
       9.94918996e-02, 5.68517716e-02, 6.40282176e-02, 5.75419652e-02,
       3.23717947e-02, 4.16071451e-02, 9.19953322e-02, 1.08812782e-01,
       4.83126114e-02, 3.83050817e-02, 6.22236086e-02, 6.60002620e-02,
      

In [ ]:
feed_data['timestamp'] = pd.to_datetime(feed_data['timestamp'])
feed_data['hour'] = feed_data.timestamp.dt.hour
feed_data.head()

In [ ]:
user_data.isnull().sum()

In [ ]:
post_text_df.isnull().sum()

In [ ]:
feed_data.isnull().sum()

In [ ]:
user_data.nunique()

In [ ]:
feed_data.nunique()

In [ ]:
post_text_df.nunique()

In [ ]:
feed_data.loc[(feed_data["action"] == "like"), "target"] = 1

In [ ]:
feed_data = feed_data.merge(user_data, on=["user_id"])
feed_data.head()

In [ ]:
feed_data = feed_data.drop("action", axis=1)

In [ ]:
import seaborn as sns
sns.catplot(x="hour", kind="count", col="target", data=feed_data, sharey=False)

In [ ]:
sns.catplot(x="gender", kind="count", col="target", data=feed_data, sharey=False)

In [ ]:
sns.catplot(x="exp_group", kind="count", col="target", data=feed_data, sharey=False)

In [ ]:
feed_data = feed_data.merge(post_text_df, on=["post_id"])

In [ ]:
feed_data.set_index("post_id")

In [ ]:
feed_data = feed_data.drop('text', axis=1)

In [ ]:
feed_data.head()

In [ ]:
feed_data = feed_data.drop("user_id", axis=1)

In [ ]:
feed_data = feed_data.drop("post_id", axis=1)

In [ ]:
feed_data.head()